In [6]:
"""
Convert with no optimization
"""

KERAS_MODEL = r'.\checkpoints\model_GRU_20'

import convert_model
# Reload
import importlib
importlib.reload(convert_model)


convert_model.convert_model(KERAS_MODEL, optimizations=None, save_path='model_no_optimizations')

Loaded model
Model: "model_GRU_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                2100      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: C:\Users\noah\AppData\Local\Temp\tmptyokschl\assets


In [37]:
import numpy as np
import os
from parameters import FileParams as fp

# Load training data
count_training = 20000
train_x = np.load(os.path.join(fp.data_dir, 'train_x.npy'))[:count_training]
train_y = np.load(os.path.join(fp.data_dir, 'train_y.npy'))[:count_training]
print(f'Percent of data positive: {np.mean(train_y) * 100:.2f}%%')
print('train_x size: ', train_x.nbytes / 1024 / 1024, 'MB')
print(len(train_x))

# Load validation data
val_x = np.load(os.path.join(fp.data_dir, 'val_x.npy'))
val_y = np.load(os.path.join(fp.data_dir, 'val_y.npy'))
print(f'Percent of data positive: {np.mean(val_y) * 100:.2f}%%')
print('val_x size: ', val_x.nbytes / 1024 / 1024, 'MB')
print(len(val_x))

Percent of data positive: 19.71%%
train_x size:  57.525634765625 MB
20000
Percent of data positive: 0.03%%
val_x size:  102.68038177490234 MB
35699


In [65]:
# Eval baseline model
import keras

keras_model: keras.Model = keras.models.load_model(KERAS_MODEL)
keras_model.summary()

Model: "model_GRU_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                2100      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________
Keras file size:  0.00390625 MB


In [41]:
# Eval on train data
loss, baseline_train_accuracy = keras_model.evaluate(train_x, train_y)
print(f'Train loss: {loss}, Train accuracy: {baseline_train_accuracy}')

625/625 [==============================] - 3s 5ms/step - loss: 0.0013 - accuracy: 0.9995
Train loss: 0.0013221293920651078, Train accuracy: 0.9994999766349792


In [42]:
# Eval on validation data
loss, baseline_val_accuracy = keras_model.evaluate(val_x, val_y)
print(f'Val loss: {loss}, Val accuracy: {baseline_val_accuracy}')

1116/1116 [==============================] - 6s 5ms/step - loss: 3.4206e-04 - accuracy: 0.9999
Val loss: 0.00034205790143460035, Val accuracy: 0.9998879432678223


In [54]:

from tqdm import tqdm
from model.modelwrapper import ModelWrapper, get_model_wrapper

def predict_all(model_wrapper: ModelWrapper, data):
    output_array = []
    for i in tqdm(range(len(data))):
        input_data = data[i]
        output_piece = model_wrapper.predict(input_data)
        output_array.append(output_piece)
    return np.array(output_array).flatten()

def get_accuracy(model_wrapper: ModelWrapper, x, y):
    predictions = predict_all(model_wrapper, x)
    predictions = np.array(predictions).flatten()
    predictions = np.round(predictions)
    return np.mean(predictions == y)

def load_and_get_accuracy(model_path):
    model_wrapper = get_model_wrapper(model_path)  # Load model
    train_accuracy = get_accuracy(model_wrapper, train_x, train_y)  # Get train accuracy
    val_accuracy = get_accuracy(model_wrapper, val_x, val_y)  # Get val accuracy
    return train_accuracy, val_accuracy, model_wrapper  # Return train acc, val acc, and model wrapper

In [55]:
no_optimizations_train_accuracy, no_optimizations_val_accuracy, no_optimizations_model = load_and_get_accuracy("model_no_optimizations.tflite")

os_name = nt


100%|██████████| 35699/35699 [00:09<00:00, 3720.51it/s]


In [61]:
import sys
file_size = os.path.getsize("model_no_optimizations.tflite") / 1024
print(f'File size: {file_size} KB')


File size: 19.25 KB
Memory size: 48 Bytes


In [47]:
train_diff = baseline_train_accuracy - no_optimizations_train_accuracy
val_diff = baseline_val_accuracy - no_optimizations_val_accuracy
print(f'Train accuracy diff: {train_diff}')
print(f'Val accuracy diff: {val_diff}')

Train accuracy diff: -2.3365020807020187e-08
Val accuracy diff: -8.775652315939908e-09


In [48]:
"""
Convert with default optimization
"""
import tensorflow as tf
convert_model.convert_model(KERAS_MODEL, optimizations=[tf.lite.Optimize.DEFAULT], save_path='model_default_optimizations')

Loaded model
Model: "model_GRU_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 20)                2100      
                                                                 
 dense_4 (Dense)             (None, 1)                 21        
                                                                 
Total params: 2,121
Trainable params: 2,121
Non-trainable params: 0
_________________________________________________________________
INFO:tensorflow:Assets written to: C:\Users\noah\AppData\Local\Temp\tmp_hpoexh2\assets


INFO:tensorflow:Assets written to: C:\Users\noah\AppData\Local\Temp\tmp_hpoexh2\assets


In [59]:
default_train_acc, default_val_acc, default_optimizations_model = load_and_get_accuracy("model_default_optimizations.tflite")
print(f'Train accuracy diff: {baseline_train_accuracy - default_train_acc}')
print(f'Val accuracy diff: {baseline_val_accuracy - default_val_acc}')

os_name = nt


100%|██████████| 35699/35699 [00:10<00:00, 3431.59it/s]

Train accuracy diff: -2.3365020807020187e-08
Val accuracy diff: -2.8020764783698304e-05


In [64]:
import sys
file_size = os.path.getsize("model_default_optimizations.tflite") / 1024
print(f'File size: {file_size} MB')

File size: 15.8828125 MB
Memory size: 48 Bytes
